In [2]:
import torch
import Utility
import SQLSentence
from torch.utils.data import Dataset, DataLoader

In [15]:
# Build Basic Data

connect = Utility.connect_to_database()
list_stock_type = (SQLSentence.QuerySQL('stock.stock_code_type', connect, ['distinct stock_type_id']))['stock_type_id'].tolist()
df_code_name = SQLSentence.QuerySQL('stock.stock_code', connect)
dict_code_name = dict(zip(df_code_name.code, df_code_name.name))
dict_type_name = ((SQLSentence.QuerySQL('stock.stock_type', connect)).drop(['id'], axis=1)).to_dict()['name']

del df_code_name

In [4]:
TRAIN_WINDOW_LENGTH = 20
TARGET_WINDOW_LENGTH = 1
THRESHOLD_VOLUME = 1000

In [27]:
df = data[data['code'] == list_unique_code_from_data[0]]

In [23]:
for type_id in list_stock_type:
    if type_id == 5: 
        data = SQLSentence.GetPriceByTypeId(list_stock_type[type_id], connect)
        data = Utility.PreprocessData(data)
        list_unique_code_from_data = data['code'].unique().tolist()

        #Todo
        # for code in list_unique_code_from_data:

SyntaxError: incomplete input (2983930336.py, line 8)

In [ ]:
class StockDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        len()